<a href="https://colab.research.google.com/github/readall/indiatranslator/blob/main/indicTranslate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install tqdm boto3 requests regex sentencepiece sacremoses transformers easyocr

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import easyocr

In [2]:
reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [4]:
image_root_path = "/content/drive/MyDrive/Colab Notebooks/"
kannada_img_file = image_root_path + 'kannada_01.jpg'
kannada_img_file_1 = '/content/drive/MyDrive/Colab Notebooks/kannada_01.jpg'

In [5]:
result = reader.readtext('kannada_02.jpg',detail = 0)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
result

['|.ಹೊಂದಿಸಿ ಬರೆಯಿರಿ:',
 '೧.ಕಾಪಾಡು',
 'ಹನಿ',
 '೨',
 'ಬುದ್ದಿವಂತ',
 'ಸಂರಕ್ಷಿಸು',
 '೩.ಮಳೆ',
 'ಆಸನ',
 '೪',
 'ಬೆಟ್ಟ',
 'ರಾಮಕೃಷ್ಣ',
 '೫.ಪೀಠ',
 'ಗುಡ್ಡ',
 'Ilಆವರಣದಲ್ಲಿ ಕೊಟ್ಟಿರುವ ಸರಿಯಾದ ಉತ್ತರವನ್ನು ಆರಿಸಿ ಬಿಟ್ಟ ಸ್ಥಳದಲ್ಲಿ ಬರೆಯಿರಿ:',
 '(ಕುವೆಂಪು, ಅಪ್ಪುಗೆ, ಪಂಡಿತರು,',
 'ತ,ಮಳೆ',
 '೧',
 '.ಕೃಷ್ಣದೇವರಾಯನ ಆಸ್ಥಾನದಲ್ಲಿ',
 'ಇದ್ದರು.',
 '೨.ನೀರಿನ ಮೂಲ',
 'ಸಂಸ್ಕೃ ತ']

In [14]:
result = reader.readtext(kannada_img_file, detail = 0)
with open('ka.txt', 'w') as f:
    f.write(str(result))

In [8]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert", keep_accents=True)

model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
%%bash
pip install mosestokenizer indic-nlp-library mock sacrebleu tensorboardX pyarrow subword-nmt
# git clone https://github.com/pytorch/fairseq.git
# cd fairseq
# pip install --editable ./
# %cd ..

Process is interrupted.


In [9]:
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [13]:
!wget https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
!unzip indic-en.zip

--2021-08-20 05:21:14--  https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 66.102.1.128, 142.251.5.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551079075 (4.2G) [application/zip]
Saving to: ‘indic-en.zip’

indic-en.zip        100%[===================>]   4.24G  53.7MB/s    in 84s     

2021-08-20 05:22:38 (51.9 MB/s) - ‘indic-en.zip’ saved [4551079075/4551079075]

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin/dict.SRC.txt  
   creating: indic-en/model/
  inflati

In [15]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 498, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 498 (delta 160), reused 132 (delta 117), pack-reused 297
Receiving objects: 100% (498/498), 1.49 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (287/287), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 3.21 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 23.2

In [16]:
!bash ./indicTrans/joint_translate.sh ka.txt en_outputs.txt 'kn' 'en' './indic-en'

Fri Aug 20 05:40:52 UTC 2021
Applying normalization and script conversion
python3: can't open file 'scripts/preprocess_translate.py': [Errno 2] No such file or directory
Number of sentences in input: 
Applying BPE
./indicTrans/joint_translate.sh: line 27: en_outputs.txt.norm: No such file or directory
python3: can't open file 'scripts/add_tags_translate.py': [Errno 2] No such file or directory
Decoding
Extracting translations, script conversion and detokenization
python3: can't open file 'scripts/postprocess_translate.py': [Errno 2] No such file or directory
Translation completed


In [17]:
# we need to be in indicTrans directory for translation model to load
%cd indicTrans/


/content/indicTrans


In [21]:
!bash joint_translate.sh /content/ka.txt en_outputs.txt 'kn' 'en' '../indic-en'

Fri Aug 20 05:45:57 UTC 2021
Applying normalization and script conversion
100% 1/1 [00:00<00:00, 29.36it/s]
Number of sentences in input: 1
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [22]:
!cat en_outputs.txt

Set to: ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', '


In [24]:
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../indic-en')

Initializing vocab and bpe
Initializing model for translation


In [25]:
indic2en_model.batch_translate(result, 'kn', 'en')

100%|██████████| 21/21 [00:00<00:00, 17472.80it/s]
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


['Write down:',
 'Save 1.',
 'a pinch of salt',
 '1, 2. (a)',
 'smart as a whip',
 'Save As',
 'Rain 3.',
 'The seat...',
 '4 (a) What does it mean to be a Christian?',
 'The mountain...',
 'Ramakrishna (cricketer)',
 'The Floss',
 'The mountain...',
 'Select the correct answer from the field below:',
 '(Great, great, great, great.)',
 'The rain...',
 '1 (a)',
 "In Krishnadevaraya's court",
 'was present.',
 'Source of water',
 'The culture']